# Mounting Google Drive since dataset is huge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Environment check

In [ ]:
import torch

try:
    print("PyTorch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU name:", torch.cuda.get_device_name(0))
except Exception as e:
    print("PyTorch not found or error:", e)
    print("Install PyTorch (see https://pytorch.org)")

In [ ]:
!ls -R "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/"

Streaming output truncated to the last 5000 lines.
1276220.jpg  1762435.jpg  2372967.jpg  2986195.jpg  3569963.jpg  672352.jpg
1276402.jpg  1769010.jpg  237586.jpg   2986318.jpg  3582510.jpg  677047.jpg
1279931.jpg  1769305.jpg  2376351.jpg  2996276.jpg  3582804.jpg  677054.jpg
1287024.jpg  1769909.jpg  2379804.jpg  3029669.jpg  358328.jpg	 681932.jpg
1296076.jpg  177116.jpg   2386723.jpg  3031009.jpg  3587714.jpg  683845.jpg
1297471.jpg  1777014.jpg  2401724.jpg  30318.jpg    3587821.jpg  687056.jpg
1297738.jpg  1780946.jpg  2402578.jpg  3040053.jpg  359542.jpg	 692573.jpg
129836.jpg   1781035.jpg  2403188.jpg  3040077.jpg  3596618.jpg  695660.jpg
1301308.jpg  1801205.jpg  2411480.jpg  304327.jpg   3598475.jpg  695875.jpg
1304025.jpg  1801285.jpg  2413992.jpg  3046102.jpg  3605239.jpg  695999.jpg
1305052.jpg  1806502.jpg  2419022.jpg  3048939.jpg  3605539.jpg  6991.jpg
1307894.jpg  1820225.jpg  2419970.jpg  3053532.jpg  3606540.jpg  700678.jpg
1310046.jpg  1821787.jpg  2430446.jpg  30

In [ ]:
import zipfile
from tqdm import tqdm
import os
import shutil

zip_path = "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/food-101.zip"
extract_to = "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/food-101_temp"
final_dir = "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/food-101"

# Step 1: Extract ZIP (skip __MACOSX and directories)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    valid_files = [f for f in zip_ref.namelist() if not f.startswith("__MACOSX") and not f.endswith('/')]
    for f in tqdm(valid_files, desc="Extracting files"):
        zip_ref.extract(f, extract_to)

# Step 2: Move contents up one level, skip hidden/system files
for item in os.listdir(extract_to):
    if item.startswith('.') or item == '__MACOSX':
        continue
    src = os.path.join(extract_to, item)
    dst = os.path.join(final_dir, item)
    shutil.move(src, dst)

# Step 3: Remove temp folder
shutil.rmtree(extract_to)

print("Extraction complete!")


Extracting files: 100%|██████████| 101010/101010 [15:30<00:00, 108.57it/s]


Extraction complete!


# Prepare Dataset and DataLoader

In [ ]:
# Import libraries
import os
import shutil
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch
import random
from tqdm import tqdm

In [ ]:
# drive_base = "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/food-101-1"
# local_base = "/content/food-101"

# images_dir = os.path.join(drive_base, "images")
# meta_dir = os.path.join(drive_base, "meta")

# # Destination folders
# train_dir = os.path.join(local_base, "train")
# test_dir  = os.path.join(local_base, "test")

# # ---------------------------
# # Function to check if folder exists and is non-empty
# # ---------------------------
# def is_nonempty_dir(path):
#     return os.path.exists(path) and len(os.listdir(path)) > 0

# # ---------------------------
# # Function to copy and organize images
# # ---------------------------
# def organize_split(split_file, dest_dir):
#     if is_nonempty_dir(dest_dir):
#         print(f"{dest_dir} already exists and is non-empty. Skipping copy.")
#         return

#     os.makedirs(dest_dir, exist_ok=True)

#     with open(split_file, "r") as f:
#         lines = f.read().splitlines()
#     for line in tqdm(lines, desc=f"Copying to {dest_dir}"):
#         class_name, img_name = line.split("/", 1)
#         src_path = os.path.join(images_dir, class_name, img_name + ".jpg")
#         class_folder = os.path.join(dest_dir, class_name)
#         os.makedirs(class_folder, exist_ok=True)
#         shutil.copy(src_path, class_folder)

# # ---------------------------
# # Copy train and test sets (only if needed)
# # ---------------------------
# organize_split(os.path.join(meta_dir, "train.txt"), train_dir)
# organize_split(os.path.join(meta_dir, "test.txt"), test_dir)

# print("Dataset preparation complete!")
drive_base = "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/food-101-1"
images_dir = os.path.join(drive_base, "images")
meta_dir = os.path.join(drive_base, "meta")

def load_split(split_file, images_dir):
    samples = []
    with open(split_file, "r") as f:
        lines = f.read().splitlines()
    for line in lines:
        class_name, img_name = line.split("/", 1)
        img_path = os.path.join(images_dir, class_name, img_name + ".jpg")
        label = class_name
        samples.append((img_path, label))
    return samples

train_samples = load_split(os.path.join(meta_dir, "train.txt"), images_dir)
test_samples  = load_split(os.path.join(meta_dir, "test.txt"),  images_dir)

classes = sorted(os.listdir(images_dir))
class_to_idx = {cls: i for i, cls in enumerate(classes)}


In [ ]:
# Define image transformations

validation_split = 0.1

train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
# Dataset class

class Food101Dataset(Dataset):
    def __init__(self, samples, class_to_idx, transform=None):
        self.samples = samples  # list of (img_path, class_name)
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, class_name = self.samples[idx]

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        label = self.class_to_idx[class_name]
        return image, label

In [ ]:
# Create datasets

# Create full train dataset
full_train_dataset = Food101Dataset(
    samples=train_samples,
    class_to_idx=class_to_idx,
    transform=train_transforms
)

# Train/Validation split
val_size = int(len(full_train_dataset) * validation_split)
train_size = len(full_train_dataset) - val_size

train_dataset, val_dataset = torch.utils.data.random_split(
    full_train_dataset,
    [train_size, val_size]
)

# Create test dataset
test_dataset = Food101Dataset(
    samples=test_samples,
    class_to_idx=class_to_idx,
    transform=test_transforms
)


In [ ]:
# Create DataLoaders

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
# Dataset info
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
print(f"Number of classes: {len(class_to_idx)}")

Number of training samples: 68175
Number of validation samples: 7575
Number of test samples: 25250
Number of classes: 102


# Model Architecture

In [ ]:
# Model architecture: ResNet50

In [ ]:
# Import libraries

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

In [ ]:
# Load pretrained ResNet50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model with pretrained weights
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 98.7MB/s]


In [ ]:
# Modify final fully connected layer

num_classes = len(class_to_idx)  # 101 for Food-101

# Original ResNet50 fc layer: nn.Linear(2048, 1000)
model.fc = nn.Linear(in_features=2048, out_features=num_classes)

# Move the model to the GPU (if available)
model = model.to(device)

# ===========================
# 3. Print model summary
# ===========================
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Training components and loop

In [ ]:
# Import libs
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Optimizer
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

In [ ]:
# Mixed precision training
scaler = GradScaler()

/tmp/ipython-input-2495738997.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.12/dist-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


In [ ]:
# Defining the training function

def train_one_epoch(model, dataloader, criterion, optimizer, device, scaler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(dataloader, leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Mixed precision context
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Backprop with scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        loop.set_description(f"Train Loss: {loss.item():.4f} Acc: {100*correct/total:.2f}%")

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
# Defining the validation function
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
# Defining the training loop
num_epochs = 1  # adjust as needed
best_val_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc*100:.2f}%")

    # Scheduler step (use validation loss)
    scheduler.step(val_loss)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/content/drive/My Drive/Master/IBU 511 - Machine Learning/Sead Project/best_model.pth")
        print("Best model saved!")


Epoch 1/1


  0%|          | 0/2131 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/tmp/ipython-input-1630042522.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Train Loss: 3.9791 Acc: 6.07%:   2%|▏         | 34/2131 [14:38<13:45:07, 23.61s/it]

# Model Evaluation & Metrics

In [ ]:
# Accuracy
def evaluate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total * 100
    return acc

test_acc = evaluate_accuracy(model, test_loader)
print(f"Test Accuracy: {test_acc:.2f}%")

In [ ]:
# Precision, Recall, F1, Classification report
from sklearn.metrics import classification_report
import numpy as np

def get_predictions(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Collecting predictions"):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return np.array(all_labels), np.array(all_preds)

true_labels, pred_labels = get_predictions(model, test_loader)

print(classification_report(
    true_labels,
    pred_labels,
    target_names=full_train_dataset.class_names
))

In [ ]:
# Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_labels, pred_labels)

plt.figure(figsize=(18, 18))
sns.heatmap(cm, cmap="Blues", xticklabels=False, yticklabels=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
# Top Misclassified Images
import random
from PIL import Image
import numpy as np

def show_misclassified(model, dataset, true, pred, max_samples=10):
    wrong_idx = np.where(true != pred)[0]
    samples = random.sample(list(wrong_idx), min(max_samples, len(wrong_idx)))

    for idx in samples:
        img_path = dataset.image_paths[idx]
        img = Image.open(img_path)

        true_class = dataset.class_names[true[idx]]
        pred_class = dataset.class_names[pred[idx]]

        plt.figure(figsize=(4,4))
        plt.imshow(img)
        plt.title(f"True: {true_class}\nPred: {pred_class}")
        plt.axis("off")
        plt.show()

show_misclassified(model, test_dataset, true_labels, pred_labels)


# Recipe Retrieval

In [ ]:
# Simple Recipe Dictionary
# ==============================================
# Step 7 — Recipe Retrieval System (Dictionary)
# ==============================================

# Example recipes for demonstration
# You will expand this with your own recipes!
recipe_db = {
    "apple_pie": {
        "title": "Classic Apple Pie",
        "ingredients": [
            "Apples",
            "Sugar",
            "Cinnamon",
            "Butter",
            "Pie crust"
        ],
        "instructions": "Mix sliced apples with sugar and cinnamon. Place into crust, cover with top layer, bake at 180°C for 50 minutes."
    },

    "pizza": {
        "title": "Homemade Pizza",
        "ingredients": [
            "Pizza dough",
            "Tomato sauce",
            "Mozzarella",
            "Olive oil",
            "Basil"
        ],
        "instructions": "Spread sauce over dough, sprinkle cheese, add toppings, bake at 220°C for 12 minutes."
    }
}

# Default fallback recipe if class doesn't exist
fallback_recipe = {
    "title": "No recipe available",
    "ingredients": [],
    "instructions": "This class has no recipe added yet. Add it in recipe_db!"
}

def get_recipe(class_name):
    """Return recipe for predicted class."""
    return recipe_db.get(class_name, fallback_recipe)

In [ ]:
get_recipe("apple_pie")

In [ ]:
# Recipes with Models prediction

In [ ]:
def predict_with_recipe(model, image_path):
    from PIL import Image
    import torch
    import torch.nn.functional as F

    model.eval()

    # Load + preprocess
    img = Image.open(image_path).convert("RGB")
    tensor = test_transforms(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(tensor)
        probs = F.softmax(outputs, dim=1)
        conf, pred_idx = torch.max(probs, 1)

    class_name = full_train_dataset.class_names[pred_idx.item()]
    confidence = conf.item() * 100

    # Get recipe
    recipe = get_recipe(class_name)

    return class_name, confidence, recipe

In [ ]:
image_path = "/content/test_image.jpg"

cls, conf, rec = predict_with_recipe(model, image_path)

print("Prediction:", cls)
print("Confidence:", round(conf, 2), "%")
print("\n--- Recipe ---")
print("Title:", rec["title"])
print("Ingredients:", rec["ingredients"])
print("Instructions:", rec["instructions"])


# Model Evaluation

In [ ]:
# ============================
# STEP 8: EVALUATE THE MODEL
# ============================

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Put model into evaluation mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 1. ACCURACY
accuracy = accuracy_score(all_labels, all_preds)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

# 2. CLASSIFICATION REPORT
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))

# 3. CONFUSION MATRIX
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

# Save the Model

In [ ]:
# ============================
# STEP 9: SAVE THE TRAINED MODEL
# ============================

model_save_path = "/content/drive/MyDrive/Master/IBU 511 - Machine Learning/Sead Project/model.pth"

# Save only the model weights (recommended)
torch.save(model.state_dict(), model_save_path)

print(f"Model saved successfully at: {model_save_path}")

# Full Inference Script

In [ ]:
# ============================
# STEP 10A: LOAD THE TRAINED MODEL
# ============================

import torch

model_load_path = "/content/drive/MyDrive/Master/IBU 511 - Machine Learning/Sead Project/model.pth"

# Recreate the model structure
loaded_model = models.resnet18(weights=None)
num_features = loaded_model.fc.in_features
loaded_model.fc = torch.nn.Linear(num_features, 101)

# Load weights
loaded_model.load_state_dict(torch.load(model_load_path, map_location=device))
loaded_model = loaded_model.to(device)
loaded_model.eval()

print("Model loaded successfully.")

from torchvision import transforms
from PIL import Image

inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

import json

with open('/content/drive/MyDrive/Master/IBU 511 - Machine Learning/Sead Project/food-101/meta/classes.txt', 'r') as f:
    classes = [line.strip() for line in f]

# ============================
# STEP 10D: INFERENCE FUNCTION
# ============================

def predict_image(image_path, model, classes):
    img = Image.open(image_path).convert("RGB")
    img_tensor = inference_transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    predicted_class = classes[predicted.item()]
    return predicted_class


test_image = "/content/drive/MyDrive/Master/IBU 511 - Machine Learning/Sead Project/test.jpg"

prediction = predict_image(test_image, loaded_model, classes)
print("Predicted:", prediction)

